In [29]:
%load_ext autoreload
%autoreload 2
%autosave 60
%matplotlib inline
from ivanocode.ivanocommon import *
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 60 seconds


In [2]:
raw = 'raw'
submissions = 'submissions'

In [3]:
df_sample_submission = pd.read_csv(f'{raw}/sample_submission.csv')
df_sample_submission.columns

Index(['id', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'],
      dtype='object')

In [4]:
df_sales_train_melt = pd.read_feather('df_sales_train_melt.feather')

In [5]:
df_sales_train_melt.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,daily_avg_count,monthly_avg_count,day_date,day_date_str,month_id,month,year,date,wm_yr_wk,sell_price,sell_price_flip_count
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0.313643,9.409305,2011-01-29,2011-01-29,1,1,2011,2011-01-29,11101,NaN,1


In [6]:
days = df_sales_train_melt['day_id'].unique()
days[0], days[-1]

(1, 1913)

In [7]:
# F1 = d_1914, ... F28 = d_1941

In [8]:
query = ("id == 'HOBBIES_1_001_CA_1_validation' or "+
         "id == 'HOBBIES_1_002_CA_1_validation' or "+
         "id == 'HOBBIES_1_004_CA_1_validation'")
subset = df_sales_train_melt.query(query).copy()

In [9]:
%%time
min_day = 1885
df_submission_melt = (df_sales_train_melt
    .groupby('id', as_index=False)
    .tail(28)
)

df_submission = (df_submission_melt
    .pivot(index='id', columns='day_id', values='sales')
    .rename(lambda x: f'F{x-min_day}', axis=1)
    .reset_index()
    .sort_values('id')
)

CPU times: user 5.48 s, sys: 2.41 s, total: 7.89 s
Wall time: 8.18 s


In [10]:
df_train = df_sales_train_melt.query('day_id < 1886').copy()

In [11]:
df_pred = df_train.query(f'day_id >= {1886-28} & day_id <= {1913-28}').copy()

In [12]:
df_valid = df_sales_train_melt.query('day_id >= 1886 & day_id <= 1913').copy()

In [13]:
aggregation_levels = {
    11: ['item_id', 'state_id'],
    10: ['item_id'],
    9:  ['store_id', 'dept_id'],
    8:  ['store_id', 'cat_id'],
    7:  ['state_id', 'dept_id'],
    6:  ['state_id', 'cat_id'],
    5:  ['dept_id'],
    4:  ['cat_id'],
    3:  ['store_id'],
    2:  ['state_id'],
    1:  [],
}
agg_level_expected_counts = {
    11: 9147,
    10: 3049,
    9:  70,
    8:  30,
    7:  21,
    6:  9,
    5:  7,
    4:  3,
    3:  10,
    2:  3
}

In [14]:
def add_group_id_col(df, grp_id, grp_fields):
    df['id'] = str(grp_id)
    if len(grp_fields) > 0:
        # Note to self: categoricals require cast
        for col in grp_fields:
            df['id'] = df['id'].str.cat(df[col], sep=':')

def with_aggregate_series(df, agg_levels=aggregation_levels):
    result = [df]
    df['agg_level'] = 12
    df['sales_$'] = df['sales']*df['sell_price']
    for grp_id, grp_fields in agg_levels.items():
        grp_11 = (df
                  .groupby(grp_fields + ['day_id'], as_index=False)
                  .agg({'sales': 'sum', 'sales_$': 'sum'})
        )
        grp_11['agg_level'] = grp_id
        add_group_id_col(grp_11, grp_id, grp_fields)

        result.append(grp_11)

    df = pd.concat(result, sort=False)
    df['sales_delta_sum'] = (df
              .groupby(['id'], as_index=False)
              ['sales']
              .transform(lambda x: x.diff().abs().sum())
    )
    # Note to self: ordering dependencies are a beast! Wouldn't it be fun to prevent cell & statement reordering unless it's safe & consistent?
    df['day_id_rel'] = df['day_id'] - df['day_id'].min()

    return df

In [15]:
df_valid_w_aggs = with_aggregate_series(df_valid)

In [16]:
df_valid_w_aggs.tail(1)

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,daily_avg_count,monthly_avg_count,...,month,year,date,wm_yr_wk,sell_price,sell_price_flip_count,agg_level,sales_$,sales_delta_sum,day_id_rel
27,1,NaN,NaN,NaN,NaN,NaN,1913,49795,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,162924.07,133244.0,27


In [17]:
df_pred_w_aggs = with_aggregate_series(df_pred)

In [18]:
df_pred_w_aggs.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,daily_avg_count,monthly_avg_count,...,month,year,date,wm_yr_wk,sell_price,sell_price_flip_count,agg_level,sales_$,sales_delta_sum,day_id_rel
56619930,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1858,0,0.313643,9.409305,...,2.0,2016.0,2016-02-29,11605.0,8.26,1.0,12,0.0,27.0,0
56619931,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1858,0,0.257710,7.731312,...,2.0,2016.0,2016-02-29,11605.0,3.97,1.0,12,0.0,9.0,0
56619932,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1858,0,0.150549,4.516466,...,2.0,2016.0,2016-02-29,11605.0,2.97,1.0,12,0.0,20.0,0


In [19]:
cols = ['id', 'day_id_rel', 'sales', 'sales_$', 'agg_level']
t = (df_valid_w_aggs[cols]
        .merge(df_pred_w_aggs[cols], 
               on=['id', 'day_id_rel'], 
               suffixes=('_valid', '_pred')))
t['daily_sales_err^2'] = (t['sales_valid'] - t['sales_pred']).pow(2)
def agg_as_is(x):
    vals = x.unique()
    if (len(vals) == 1):
        return vals[0]
    raise ValueError(f"Expected single unique val, got: {vals}" )

t = t.groupby(['id', 'agg_level_valid'], as_index=False).agg({
    'sales_valid': lambda x: x.diff().pow(2).sum(),
    'daily_sales_err^2': lambda x: x.sum(),
    'sales_$_valid': 'sum'
}).rename({
    'sales_valid': 'sales_diff^2_sum',
    'daily_sales_err^2': 'sales_err^2_sum'
}, axis=1)
t['agg_weight'] = t.groupby('agg_level_valid')['sales_$_valid'].transform(lambda x: x.sum())
t['series_weight'] = t['sales_$_valid']/t['agg_weight']

In [20]:
diff_sum_squared = lambda x: x.diff().pow(2).sum()
diff_sum_squared.__name__ = 'diff_sum_squared'
sales_diff_summs = (df_train
    .groupby(['id'], as_index=False)
    .agg({
        'sales': [
            diff_sum_squared,
            'count'],
    })
    .pipe(
        lambda df: drop_level(df, level=1, axis=1)
    ).rename({
        'sales_count': 'trn_sales_count'
    }, axis=1)
)

In [21]:
sales_diff_summs.head(1)

,id,sales_diff_sum_squared,trn_sales_count
0,FOODS_1_001_CA_1_validation,5463,1885


In [22]:
trn_days_denom = sales_diff_summs['trn_sales_count'].unique()
assert [1885] == trn_days_denom, f"Expected {trn_days_denom} days of sales to be used in denom sum, got {trn_days_denom}"

In [23]:
t = t.merge(sales_diff_summs, on='id', validate='one_to_one')

In [24]:
h = 28
t['rmsse'] = ((
                (t['trn_sales_count']-1.0) * t['sales_err^2_sum'])/
                (h * t['sales_diff^2_sum'])
             ).pow(0.5)
# TODO: rmsse est for series with no sales - will not be a problem when coming from trn
t.loc[t['sales_$_valid'] ==0, 'rmsse'] = 0
t['wrmsse'] = t['rmsse']*t['series_weight']

In [25]:
# that's underestimation of 0.83770 from the submission, plus I've got a bunch of todos above to address
# 0.7456401780934122 - when calculated using validation set days in the denom :/
# 0.7472960893819334 - latest before unit testing 
# 0.8626966331787638 - after unit test impl, 90 secs of calc time is a bit too much :/
wrmsse_total = t['wrmsse'].sum()/12
wrmsse_total

0.7472960893819334

In [26]:
# df_train_w_aggs = with_aggregate_series(df_train)

In [27]:
# TODO: wrmsse_total(df_train_w_aggs, df_valid_w_aggs, df_pred_w_aggs)

In [30]:
example_agg_levels = {
    1: []
}

def generate_series(display=no_op):
    import numpy as np
    series_len = 10
    h = pred_len = 4
    pred_sales_a = 10.0
    pred_sales_b = 12.0

    example_a = pd.DataFrame({
        'id':         np.tile(['A'],                       reps=series_len),
        'sales':      np.tile([1, 0],                      reps=series_len//2),
        'sell_price': np.tile([pred_sales_a*2/pred_len],   reps=series_len),
        'day_id':     np.arange(0,                              series_len, step=1)
    })
    example_b = pd.DataFrame({
        'id':         np.tile(['B'],                         reps=series_len),
        'sales':      np.tile([0, 2],                        reps=series_len//2),
        'sell_price': np.tile([pred_sales_b*2/(pred_len*2)], reps=series_len),
        'day_id':     np.arange(0,                                series_len, step=1)
    })
    example = pd.concat([example_a, example_b])
    example.sort_values(['day_id', 'id'], inplace=True)

    display("total_sales", example
        .eval('sales_d = sell_price*sales')
        .groupby('id')
        .tail(h)
        .groupby('id')
        .agg({'sales_d': 'sum'})
    )
    display("total_counts", example.groupby('id').count())

    trn_len = 2*(series_len-h)
    example_trn = example[:trn_len].copy()
    example_val = example[trn_len-2*series_len:].copy()
    len(example_trn), len(example_val)
    display("split_counts", (pd.concat([example_trn, example_val], 
          sort=False, 
          keys=['trn', 'val'])
         .reset_index()
         .groupby(['level_0', 'id'], sort=False)
         .count()
        ))
    
    example_val_w_aggs = with_aggregate_series(example_val, agg_levels = example_agg_levels)
    inversed_sales = (example_val
        .groupby('id')['sales']
        .transform(lambda x: x.max() - x)
    )
    example_val_inv = example_val.copy()
    example_val_inv['sales'] = inversed_sales
    example_val_inv['sales_$'] = example_val_inv['sales']*example_val_inv['sell_price']
    example_val_inv_w_aggs = with_aggregate_series(example_val_inv, agg_levels = example_agg_levels)
    example_trn_w_aggs =     with_aggregate_series(example_trn,     agg_levels = example_agg_levels)

    res = dict()
    res['val_inv'] = example_val_inv
    res['trn_w_aggs'] = example_trn_w_aggs
    res['val_w_aggs'] = example_val_w_aggs
    res['val_inv_w_aggs'] = example_val_inv_w_aggs
    return res

In [31]:
def wrmsse_total(df_train, df_valid_w_aggs, df_pred_w_aggs, display=no_op):
    cols = ['id', 'day_id_rel', 'sales', 'sales_$', 'agg_level']
    t = (df_valid_w_aggs[cols]
            .merge(df_pred_w_aggs[cols], 
                   on=['id', 'day_id_rel'], 
                   suffixes=('_valid', '_pred')))
    t['daily_sales_err^2'] = (t['sales_valid'] - t['sales_pred']).pow(2)
    display("t1", t)

    t = t.groupby(['id', 'agg_level_valid'], as_index=False).agg({
        'daily_sales_err^2': lambda x: x.sum(),
        'sales_$_valid': 'sum'
    }).rename({
        'daily_sales_err^2': 'sales_err^2_sum'
    }, axis=1)
    display("t2", t)
    t['agg_weight'] = (t.groupby('agg_level_valid')
        ['sales_$_valid']
       .transform(lambda x: x.sum())
    )
    t['series_weight'] = t['sales_$_valid']/t['agg_weight']

    diff_sum_squared = lambda x: x.diff().pow(2).sum()
    diff_sum_squared.__name__ = 'diff_sum_squared'
    sales_diff_summs = (df_train
        .groupby(['id'], as_index=False)
        .agg({
            'sales': [
                diff_sum_squared,
                'count'],
        })
        .pipe(
            lambda df: drop_level(df, level=1, axis=1)
        ).rename({
            'sales_count': 'trn_sales_count',
            'sales_diff_sum_squared': 'trn_diff_sum_squared'
        }, axis=1)
    )
    display("sales_diff_summs", sales_diff_summs)

    # TODO: this sort of ignores aggs unless they're there
    t = t.merge(sales_diff_summs, on='id', how='left', validate='one_to_one')
    display("t3", t)
    
    h = df_valid_w_aggs['day_id_rel'].nunique()
    n_aggs = t['agg_level_valid'].nunique()
    t['rmsse'] = ((
                    (t['trn_sales_count']-1.0) * t['sales_err^2_sum'])/
                    (h * t['trn_diff_sum_squared'])
                 ).pow(0.5)
    display("t4", t)

    # TODO: rmsse est for series with no sales - will not be a problem when coming from trn
    # TODO: failing fast might also be a better idea than hiding inconsistent test data 
    t.loc[t['sales_$_valid'] == 0, 'rmsse'] = 0
    t['wrmsse'] = t['rmsse']*t['series_weight']

    wrmsse_total = t['wrmsse'].sum()/n_aggs
    return wrmsse_total

example = generate_series()
perfect_score = wrmsse_total(example['trn_w_aggs'], 
                             example['val_w_aggs'],
                             example['val_w_aggs'])
assert 0.0 == perfect_score

score = wrmsse_total(example['trn_w_aggs'], 
                     example['val_inv_w_aggs'],
                     example['val_w_aggs'])
assert 1.0 == score

score = wrmsse_total(example['trn_w_aggs'], 
                     example['val_w_aggs'],
                     example['val_inv_w_aggs'])
assert 1.0 == score

In [32]:
import gc
gc.set_debug(0)
gc.collect()

280

In [33]:
agg_series_counts = (df_valid_w_aggs.query('day_id == 1886')
    [['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'agg_level', 'state_id']]
    .groupby(['agg_level'], as_index=False)
    .count()
)

In [34]:
df_valid_w_aggs.query('day_id==1909')['id'].count()

42840

In [35]:
grp_fields=['state_id', 'cat_id']
denom = lambda x: x.diff().abs().sum()
denom.__name__ = 'a'
grp_11 = (df_valid
    .groupby(grp_fields + ['day_id'], as_index=False)
    .agg({'sales': 'sum', 'sales_$': 'sum'})
)
grp_11.head()

,state_id,cat_id,day_id,sales,sales_$
0,CA,FOODS,1886,9821,27349.95
1,CA,FOODS,1887,9845,26741.79
2,CA,FOODS,1888,9165,24731.91
3,CA,FOODS,1889,9941,26434.36
4,CA,FOODS,1890,11891,32464.87


In [36]:
pd.__version__

'0.25.0'

In [37]:
df_valid.query('id == "HOBBIES_1_001_CA_1_validation"')['sales'].diff().abs().sum()

30.0

In [38]:
%%time
for level, expected_count in agg_level_expected_counts.items():
    actual_count = (agg_series_counts
         .query(f'agg_level == {level}')
         .drop('agg_level', axis=1)
         .max(axis=1)
         .values[0]
    )
    assert expected_count == actual_count, f"Expected {expected_count} at level {level}, got: {actual_count}"

CPU times: user 35.9 ms, sys: 1.5 ms, total: 37.4 ms
Wall time: 37.3 ms


In [39]:
# grp_11['id'] = grp_11.sum(axis=1) # fishy, ints and not they sorta collide
"""
>>> df_valid_w_aggs.query('day_id == 1886')['id'].value_counts()[:3]
1897    3041
1898    1952
1899    1394
Name: id, dtype: int64
"""

"\n>>> df_valid_w_aggs.query('day_id == 1886')['id'].value_counts()[:3]\n1897    3041\n1898    1952\n1899    1394\nName: id, dtype: int64\n"

In [40]:
# TODO:
# pre-aggregate series
# add weight per sales volumes
# calculate rmssse
# calculate wrmsse
# ...
# Profit!

In [41]:
eval_rows = df_sample_submission[df_sample_submission['id'].str.contains('_evaluation')]

In [42]:
(pd.concat([df_submission, eval_rows])
    .to_csv(f'{submissions}/0200-naive.csv', index=False)
)

In [43]:
!head -n2 {submissions}/0200-naive.csv

id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
FOODS_1_001_CA_1_validation,2,1,1,0,4,0,0,4,1,3,0,1,0,2,2,0,1,1,0,2,0,4,1,1,0,1,1,0


In [44]:
!head -n2 {raw}/sample_submission.csv

id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
!wc -l {submissions}/0200-naive.csv

   60981 submissions/0200-naive.csv


In [ ]:
!wc -l {raw}/sample_submission.csv

   60981 raw/sample_submission.csv


In [ ]:
!open {submissions}

In [ ]:
!open https://www.kaggle.com/c/m5-forecasting-accuracy/submit